# Central-bank rate changes (G20)  
© 2026 Jim Domeij  

Each month, we count how many G20 central banks **raised** their policy rate (hikes) and how many **lowered** it (cuts), as a simple proxy for the **breadth** of global tightening vs easing.

- **Hikes**: number of central banks raising rates  
- **Cuts**: number of central banks cutting rates  
- **Net**: `hikes − cuts` (positive = broad tightening, negative = broad easing)

The top panel shows hikes (up) and cuts (down). The bottom panel shows the net balance on the same scale.

*Note: This is a **count of central banks** changing rates, not the **magnitude** of the rate moves.*


In [1]:
import json
from pathlib import Path

import altair as alt

alt.renderers.set_embed_options(actions=False)

chart_file = Path("docs/examples/macro") / "hikes-cuts-count-hero.json"
with open(chart_file) as f:
    spec = json.load(f)
alt.Chart.from_dict(spec)

alt.VConcatChart(...)

In [2]:
from polars_bloomberg import BQuery

query = """
let(
    #ref_date = policy_rate().period_reference_date;
    #policy_rate_changes = diff(dropNA(policy_rate(), remove_id=True));
    #count_cuts = sum(group(if(#policy_rate_changes < 0, 1, 0), by=#ref_date));
    #count_hikes = sum(group(if(#policy_rate_changes > 0, 1, 0), by=#ref_date));
    #net_hike_count = #count_hikes - #count_cuts;
)
get(#count_cuts, #count_hikes, #net_hike_count)
for(countries('G20'))
with(pr=range(2016M12, 2025M12), pt=M, act_est_data=A, fill=PREV)
preferences(dropCols=["ORIG_IDS"])
"""

with BQuery() as bq:
    df = bq.bql(query).combine()
df.head(3)

ID,#count_cuts,#REF_DATE,#count_hikes,#net_hike_count
str,f64,date,f64,f64
"""2016-12-31T00-00-00Z""",0.0,2016-12-31,0.0,0.0
"""2017-01-31T00-00-00Z""",1.0,2017-01-31,0.0,-1.0
"""2017-02-28T00-00-00Z""",1.0,2017-02-28,1.0,0.0


In [4]:
import altair as alt
import polars as pl

g = df.select(
    pl.col("#REF_DATE").alias("date"),
    pl.col("#count_hikes").alias("hikes"),
    (pl.col("#count_cuts") * -1).alias("cuts"),
    pl.col("#net_hike_count").alias("net"),
)

max_abs = (int(max(g["hikes"].max(), -g["cuts"].min())) + 1) // 2 * 2

hover = alt.selection_point(
    fields=["date"],
    nearest=True,
    on="mouseover",
    empty=False,
    clear="mouseout",
    name="hover",
)

x_enc = alt.X(
    "date:T",
    timeUnit="yearmonth",
    axis=alt.Axis(title=None, format="%Y", tickCount="year", labelAngle=0, grid=False),
)

# --- Top panel ---
bars = (
    alt.Chart(g)
    .transform_fold(["hikes", "cuts"], as_=["type", "count"])
    .mark_bar()
    .encode(
        x=x_enc,
        y=alt.Y(
            "count:Q",
            scale=alt.Scale(domain=[-max_abs, max_abs]),
            axis=alt.Axis(title="Count", tickMinStep=2),
        ),
        color=alt.Color(
            "type:N",
            scale=alt.Scale(domain=["hikes", "cuts"], range=["#2563EB", "#F97316"]),
            legend=alt.Legend(title=None, orient="top"),
        ),
        # FIX 1: use selection object, not "hover" string
        opacity=alt.condition(hover, alt.value(1.0), alt.value(0.60)),
        tooltip=[
            alt.Tooltip("date:T", title="Month", format="%b %Y"),
            alt.Tooltip("type:N", title="Type"),
            alt.Tooltip("count:Q", title="Count"),
        ],
    )
)

zero_rule = alt.Chart(g).mark_rule(color="#444").encode(y=alt.datum(0))

# FIX 2: axis=None so this layer cannot reintroduce x-gridlines
v_rule = (
    alt.Chart(g)
    .mark_rule(color="#999")
    .encode(x=alt.X("date:T", timeUnit="yearmonth", axis=None))
    .transform_filter(hover)  # FIX 1 also here
)

top = (bars + zero_rule + v_rule).properties(height=260, width=600)

# --- Bottom panel ---
net_base = alt.Chart(g).encode(x=x_enc)

net_pos = (
    net_base.transform_calculate(net_pos="max(datum.net, 0)")
    .mark_area(opacity=0.35)
    .encode(
        y=alt.Y(
            "net_pos:Q",
            scale=alt.Scale(domain=[-max_abs, max_abs]),
            axis=alt.Axis(title="Net (hikes − cuts)", tickMinStep=2),
        ),
        color=alt.value("#2563EB"),
        tooltip=[
            alt.Tooltip("date:T", title="Month", format="%b %Y"),
            alt.Tooltip("net:Q", title="Net"),
        ],
    )
)

net_neg = (
    net_base.transform_calculate(net_neg="min(datum.net, 0)")
    .mark_area(opacity=0.35)
    .encode(y="net_neg:Q", color=alt.value("#F97316"))
)

net_dot = net_base.mark_point(filled=True, size=30, color="#111").encode(
    y=alt.Y("net:Q", scale=alt.Scale(domain=[-max_abs, max_abs]), axis=None),
    opacity=alt.condition(hover, alt.value(1.0), alt.value(0.35)),
    tooltip=[
        alt.Tooltip("date:T", title="Month", format="%b %Y"),
        alt.Tooltip("net:Q", title="Net"),
    ],
)

bottom = (net_pos + net_neg + zero_rule + net_dot + v_rule).properties(
    height=140, width=600
)

chart = (
    alt.vconcat(top, bottom, spacing=8)
    .add_params(hover)  # add once, shared across panels
    .resolve_scale(x="shared")
    .properties(
        title=alt.TitleParams(
            text="Central bank rate changes",
            subtitle="Monthly hikes (up) and cuts (down); net balance shown below",
            anchor="start",
            fontSize=16,
            subtitleFontSize=12,
        )
    )
    .configure_view(stroke=None)
    # extra safety: kill x-grid globally
    .configure_axisX(grid=False)
    .configure_axis(labelFontSize=11, titleFontSize=12)
    .configure_axisY(grid=True, gridOpacity=0.20)
    .configure_legend(labelFontSize=11)
)

chart.save(chart_file)
chart.save(chart_file.with_suffix(".png"), scale_factor=0.75)
